# Lab 2.5.2: Pipeline Showcase

**Module:** 2.5 - Hugging Face Ecosystem  
**Time:** 2 hours  
**Difficulty:** ⭐⭐ (Intermediate)

---

## Learning Objectives

By the end of this lab, you will:
- [ ] Master the `pipeline()` API for rapid prototyping
- [ ] Use 5 different pipeline types (sentiment, NER, QA, summarization, text-generation)
- [ ] Understand when to use pipelines vs. manual model loading
- [ ] Measure and compare pipeline performance

---

## Prerequisites

- Completed: Lab 2.5.1 (Hub Exploration)
- Knowledge of: Basic NLP concepts (tokens, embeddings)

---

## DGX Spark Optimization Notes

When using pipelines on DGX Spark (128GB unified memory, Blackwell GB10):
- Always use `torch_dtype=torch.bfloat16` for GPU inference (native Blackwell support)
- Use `device=0` for GPU (not `device="cuda"`)
- Clean up between pipelines: `del pipeline; gc.collect(); torch.cuda.empty_cache()`
- For production inference, consider Module 3.3 (SGLang, vLLM) for higher throughput

---

## Real-World Context

**The Swiss Army Knife of AI**: Imagine you're building a customer support dashboard. You need to:
- Analyze sentiment of incoming messages
- Extract key entities (product names, dates, prices)
- Answer common questions automatically
- Summarize long customer complaints
- Generate response drafts

With **Hugging Face Pipelines**, you can prototype ALL of these in under 10 minutes!

---

## ELI5: What is a Pipeline?

> **Imagine you want to make a smoothie...**
>
> The hard way: Get fruits, peel them, chop them, put in blender, blend at right speed, pour into glass.
>
> The pipeline way: Put everything in a smoothie maker with one button - it does ALL the steps automatically!
>
> **In AI terms:** A pipeline wraps tokenization + model loading + inference + post-processing into a single function call:
> ```python
> # Instead of 50 lines of code...
> result = pipeline("sentiment-analysis")("I love this!")
> # Output: [{'label': 'POSITIVE', 'score': 0.9998}]
> ```
>
> It's the fastest way to go from "I have text" to "I have AI predictions"!

---

## Part 1: Setup and Basics

In [ ]:
import torch
from transformers import pipeline
import time
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# Check environment
print("Environment Check")
print("=" * 50)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Set device for pipelines
DEVICE = 0 if torch.cuda.is_available() else -1
print(f"\nUsing device: {'GPU' if DEVICE >= 0 else 'CPU'}")

### 1.1 Your First Pipeline

Let's start with the simplest possible example:

In [ ]:
# The simplest AI application - ONE line of code!
classifier = pipeline("sentiment-analysis", device=DEVICE)

# Classify a single text
result = classifier("I love learning about AI! This is so exciting!")
print(result)

### What Just Happened?

In that ONE line, the pipeline:
1. Downloaded a pre-trained model (distilbert-base-uncased-finetuned-sst-2-english)
2. Downloaded the matching tokenizer
3. Tokenized your input text
4. Ran inference on the model
5. Converted logits to probabilities
6. Returned a human-readable result

That's what we mean by "Swiss Army Knife"!

---

## Part 2: The Five Essential Pipelines

### Pipeline 1: Sentiment Analysis (Text Classification)

In [ ]:
print("="*70)
print("PIPELINE 1: SENTIMENT ANALYSIS")
print("="*70)
print("\nUse case: Understanding if customer feedback is positive or negative")
print("-"*70)

# Create sentiment pipeline (already loaded above, but let's be explicit)
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=DEVICE,
    torch_dtype=torch.bfloat16 if DEVICE >= 0 else torch.float32
)

# Test examples
test_reviews = [
    "This product exceeded my expectations! Absolutely wonderful!",
    "Terrible experience. The product arrived broken and customer service was unhelpful.",
    "It's okay. Does what it's supposed to do, nothing more.",
    "Best purchase I've made this year! Highly recommend!",
    "Waste of money. Don't buy this."
]

print("\nResults:")
print("-"*70)
for review in test_reviews:
    result = sentiment_analyzer(review)[0]
    emoji = "" if result['label'] == 'POSITIVE' else ""
    print(f"{emoji} [{result['label']:8}] ({result['score']:.1%}) | {review[:50]}...")

# Batch processing (more efficient)
print("\n\nBatch Processing (more efficient):")
start = time.time()
batch_results = sentiment_analyzer(test_reviews)
print(f"Processed {len(test_reviews)} reviews in {(time.time()-start)*1000:.1f}ms")

### Pipeline 2: Named Entity Recognition (NER)

In [ ]:
print("\n" + "="*70)
print("PIPELINE 2: NAMED ENTITY RECOGNITION (NER)")
print("="*70)
print("\nUse case: Extract names, organizations, locations from text")
print("-"*70)

# Clean up previous model
del sentiment_analyzer
torch.cuda.empty_cache() if DEVICE >= 0 else None

# Create NER pipeline
ner = pipeline(
    "ner",
    model="dslim/bert-base-NER",
    aggregation_strategy="simple",  # Groups tokens into entities
    device=DEVICE
)

# Test examples
test_texts = [
    "Apple CEO Tim Cook announced the new iPhone 15 at the Cupertino headquarters.",
    "Elon Musk's company SpaceX launched a Falcon 9 rocket from Cape Canaveral, Florida.",
    "The European Union and United States signed a trade agreement in Brussels."
]

print("\nResults:")
for text in test_texts:
    print(f"\nText: {text}")
    print("Entities:")
    entities = ner(text)
    for ent in entities:
        print(f"  - {ent['entity_group']:12} : '{ent['word']}' (confidence: {ent['score']:.1%})")

### Pipeline 3: Question Answering

In [ ]:
print("\n" + "="*70)
print("PIPELINE 3: QUESTION ANSWERING")
print("="*70)
print("\nUse case: Extract answers from documents (extractive QA)")
print("-"*70)

# Clean up
del ner
torch.cuda.empty_cache() if DEVICE >= 0 else None

# Create QA pipeline
qa = pipeline(
    "question-answering",
    model="deepset/roberta-base-squad2",
    device=DEVICE
)

# Context about DGX Spark (imagine this is a product document)
context = """
The NVIDIA DGX Spark is a personal AI supercomputer designed for developers 
and researchers. It features the Blackwell GB10 Superchip with 128GB of unified 
LPDDR5X memory shared between CPU and GPU. The system delivers 1 PFLOP of NVFP4 
AI performance and can run models up to 200 billion parameters. It was announced 
at CES 2025 and is expected to ship in May 2025 with a starting price around $3,000.
"""

questions = [
    "How much memory does the DGX Spark have?",
    "What is the price of DGX Spark?",
    "When will DGX Spark ship?",
    "How many parameters can it handle?"
]

print("\nContext: (DGX Spark product info)")
print("-"*70)
print(context.strip())
print("\nQuestions & Answers:")
print("-"*70)

for question in questions:
    result = qa(question=question, context=context)
    print(f"Q: {question}")
    print(f"A: {result['answer']} (confidence: {result['score']:.1%})")
    print()

### Pipeline 4: Text Summarization

In [ ]:
print("\n" + "="*70)
print("PIPELINE 4: TEXT SUMMARIZATION")
print("="*70)
print("\nUse case: Condense long documents into key points")
print("-"*70)

# Clean up
del qa
torch.cuda.empty_cache() if DEVICE >= 0 else None

# Create summarization pipeline
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=DEVICE,
    torch_dtype=torch.bfloat16 if DEVICE >= 0 else torch.float32
)

# Long article to summarize
article = """
Artificial intelligence has transformed the technology landscape in unprecedented ways 
over the past decade. Large language models, which can understand and generate human-like 
text, have become increasingly powerful and accessible. Companies like OpenAI, Google, 
and Anthropic have released models capable of writing code, answering complex questions, 
and even passing professional exams.

The implications of these advances extend far beyond the tech industry. Healthcare 
professionals are using AI to diagnose diseases and develop new treatments. Financial 
institutions employ machine learning algorithms to detect fraud and make investment 
decisions. Even creative fields like art and music are being influenced by generative 
AI tools that can produce original works.

However, these developments have also raised significant concerns. Issues around job 
displacement, privacy, bias in AI systems, and the potential misuse of the technology 
have prompted calls for regulation and ethical guidelines. Many experts argue that 
while AI offers tremendous benefits, society must carefully consider how to deploy 
these powerful tools responsibly.
"""

print(f"Original article ({len(article.split())} words):")
print("-"*70)
print(article.strip())

print("\n\nSummary:")
print("-"*70)
summary = summarizer(
    article,
    max_length=100,
    min_length=30,
    do_sample=False
)
print(summary[0]['summary_text'])
print(f"\n(Reduced to {len(summary[0]['summary_text'].split())} words)")

### Pipeline 5: Text Generation

In [ ]:
print("\n" + "="*70)
print("PIPELINE 5: TEXT GENERATION")
print("="*70)
print("\nUse case: Generate creative text, code, or continue a prompt")
print("-"*70)

# Clean up
del summarizer
torch.cuda.empty_cache() if DEVICE >= 0 else None

# Create text generation pipeline
generator = pipeline(
    "text-generation",
    model="gpt2-medium",  # Larger than gpt2-small, still fits easily on DGX Spark
    device=DEVICE,
    torch_dtype=torch.bfloat16 if DEVICE >= 0 else torch.float32
)

# Generation prompts
prompts = [
    "The future of artificial intelligence is",
    "In a world where robots and humans coexist,",
    "The most important skill for the 21st century is"
]

print("\nGenerations:")
print("-"*70)

for prompt in prompts:
    print(f"\nPrompt: '{prompt}'")
    print("Generated:")
    
    result = generator(
        prompt,
        max_new_tokens=50,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=generator.tokenizer.eos_token_id
    )
    
    # Show continuation (removing the prompt)
    full_text = result[0]['generated_text']
    continuation = full_text[len(prompt):]
    print(f"  {continuation.strip()}")
    print()

---

## Part 3: Pipeline Performance Comparison

Let's measure and compare the performance of our pipelines:

In [ ]:
# Clean up previous pipeline
del generator
torch.cuda.empty_cache() if DEVICE >= 0 else None

def benchmark_pipeline(
    task: str,
    model: str,
    test_input: Any,
    warmup_runs: int = 3,
    test_runs: int = 10
) -> Dict[str, float]:
    """
    Benchmark a pipeline's performance.
    
    Returns dict with load_time, inference_time, memory_gb
    """
    torch.cuda.empty_cache() if DEVICE >= 0 else None
    initial_memory = torch.cuda.memory_allocated() / 1e9 if DEVICE >= 0 else 0
    
    # Time loading
    start = time.time()
    pipe = pipeline(
        task,
        model=model,
        device=DEVICE,
        torch_dtype=torch.bfloat16 if DEVICE >= 0 else torch.float32
    )
    load_time = time.time() - start
    
    memory_used = (torch.cuda.memory_allocated() / 1e9 - initial_memory) if DEVICE >= 0 else 0
    
    # Warmup
    for _ in range(warmup_runs):
        _ = pipe(test_input)
    
    # Benchmark inference
    if DEVICE >= 0:
        torch.cuda.synchronize()
    
    times = []
    for _ in range(test_runs):
        start = time.time()
        _ = pipe(test_input)
        if DEVICE >= 0:
            torch.cuda.synchronize()
        times.append((time.time() - start) * 1000)
    
    # Clean up
    del pipe
    torch.cuda.empty_cache() if DEVICE >= 0 else None
    
    return {
        "load_time_s": load_time,
        "inference_mean_ms": sum(times) / len(times),
        "inference_std_ms": (sum((t - sum(times)/len(times))**2 for t in times) / len(times)) ** 0.5,
        "memory_gb": memory_used,
        "throughput_per_sec": 1000 / (sum(times) / len(times))
    }

# Benchmark configurations
benchmarks = [
    ("sentiment-analysis", "distilbert-base-uncased-finetuned-sst-2-english", "This is a test sentence."),
    ("ner", "dslim/bert-base-NER", "Apple CEO Tim Cook announced new products."),
    ("question-answering", "deepset/roberta-base-squad2", {"question": "What is AI?", "context": "AI is artificial intelligence."}),
]

print("\nPIPELINE BENCHMARKS")
print("="*80)
print(f"{'Task':<25} {'Load(s)':<10} {'Infer(ms)':<12} {'Memory(GB)':<12} {'Throughput':>12}")
print("-"*80)

results = []
for task, model, test_input in benchmarks:
    print(f"Benchmarking {task}...", end=" ", flush=True)
    metrics = benchmark_pipeline(task, model, test_input)
    results.append((task, metrics))
    print(f"\r{task:<25} {metrics['load_time_s']:<10.2f} {metrics['inference_mean_ms']:<12.2f} {metrics['memory_gb']:<12.2f} {metrics['throughput_per_sec']:>10.1f}/s")

---

## Part 4: Advanced Pipeline Usage

### 4.1 Zero-Shot Classification

Classify text into categories the model has never seen during training!

In [ ]:
print("\n" + "="*70)
print("BONUS: ZERO-SHOT CLASSIFICATION")
print("="*70)
print("\nClassify text into ANY categories - no training needed!")
print("-"*70)

# Create zero-shot classifier
zero_shot = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=DEVICE
)

# Custom categories
categories = ["technology", "sports", "politics", "entertainment", "business"]

test_headlines = [
    "NVIDIA announces new AI chip with breakthrough performance",
    "Lakers defeat Celtics in overtime thriller",
    "Senate passes new infrastructure bill with bipartisan support",
    "Marvel announces new superhero movie for 2025",
    "Tech startup raises $500 million in Series C funding"
]

print(f"\nCategories: {categories}")
print("\nResults:")
print("-"*70)

for headline in test_headlines:
    result = zero_shot(headline, candidate_labels=categories)
    top_label = result['labels'][0]
    top_score = result['scores'][0]
    print(f"[{top_label:12}] ({top_score:.1%}) | {headline}")

# Clean up
del zero_shot
torch.cuda.empty_cache() if DEVICE >= 0 else None

### 4.2 Fill-Mask (Understanding Language Models)

In [ ]:
print("\n" + "="*70)
print("BONUS: FILL-MASK")
print("="*70)
print("\nSee how language models understand context!")
print("-"*70)

# Create fill-mask pipeline
fill_mask = pipeline("fill-mask", model="bert-base-uncased", device=DEVICE)

# Test sentences with [MASK]
masked_sentences = [
    "The capital of France is [MASK].",
    "I went to the [MASK] to buy groceries.",
    "The doctor prescribed [MASK] for my headache.",
    "Python is a popular [MASK] language."
]

for sentence in masked_sentences:
    print(f"\nInput: {sentence}")
    results = fill_mask(sentence)
    print("Top predictions:")
    for r in results[:3]:
        print(f"  - '{r['token_str']}' ({r['score']:.1%})")

# Clean up
del fill_mask
torch.cuda.empty_cache() if DEVICE >= 0 else None

---

## Part 5: When NOT to Use Pipelines

Pipelines are great for prototyping, but sometimes you need more control:

In [ ]:
print("\nPIPELINE vs MANUAL LOADING")
print("="*70)

comparison = """
USE PIPELINE WHEN:
  Quick prototyping and experimentation
  Standard tasks with default settings
  Single-model inference
  Batch processing with simple inputs

USE MANUAL LOADING WHEN:
  Need custom preprocessing/postprocessing
  Fine-tuning or training
  Need access to hidden states or attention weights
  Complex multi-model pipelines
  Maximum performance optimization
  Need to modify model architecture
"""
print(comparison)

# Example: Manual loading gives you more control
from transformers import AutoTokenizer, AutoModelForSequenceClassification

print("\nExample: Manual Loading for More Control")
print("-"*70)

# Manual loading
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
).to("cuda" if DEVICE >= 0 else "cpu")

# Now you can access everything!
text = "This is an amazing product!"
inputs = tokenizer(text, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True, output_attentions=True)

print(f"Logits shape: {outputs.logits.shape}")
print(f"Number of hidden states: {len(outputs.hidden_states)}")
print(f"Number of attention layers: {len(outputs.attentions)}")
print(f"Attention shape (per layer): {outputs.attentions[0].shape}")

# You can analyze attention patterns, extract embeddings, etc.
print("\nWith manual loading, you can:")
print("  - Access all hidden states for each layer")
print("  - Visualize attention patterns")
print("  - Extract embeddings for downstream tasks")
print("  - Modify inference behavior")

# Clean up
del model, tokenizer
torch.cuda.empty_cache() if DEVICE >= 0 else None

---

## Try It Yourself: Build a Multi-Pipeline App

Combine multiple pipelines to analyze customer feedback:

**Your Task:** Complete the `analyze_customer_feedback()` function to:
1. **Sentiment Analysis** - Determine if the feedback is positive or negative
2. **Named Entity Recognition** - Extract product names, people, organizations, locations
3. **Summarization** - If text is long (> 100 words), provide a summary

<details>
<summary>💡 Hint: Pipeline Creation</summary>

```python
# Create each pipeline (remember to clean up between them!)
sentiment_pipe = pipeline("sentiment-analysis", device=DEVICE)
ner_pipe = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple", device=DEVICE)
summarizer_pipe = pipeline("summarization", device=DEVICE)
```
</details>

<details>
<summary>💡 Hint: Checking Text Length</summary>

```python
# Count words in text
word_count = len(text.split())
if word_count > 100:
    # Summarize long text
    summary = summarizer_pipe(text, max_length=50, min_length=20)[0]['summary_text']
```
</details>

<details>
<summary>💡 Hint: Complete Solution Structure</summary>

```python
def analyze_customer_feedback(text: str) -> Dict[str, Any]:
    result = {"original_text": text, "sentiment": None, "entities": [], "summary": None}
    
    # 1. Sentiment
    sentiment_pipe = pipeline("sentiment-analysis", device=DEVICE)
    sentiment_result = sentiment_pipe(text)[0]
    result["sentiment"] = {"label": sentiment_result["label"], "score": sentiment_result["score"]}
    del sentiment_pipe; torch.cuda.empty_cache()
    
    # 2. NER
    ner_pipe = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple", device=DEVICE)
    entities = ner_pipe(text)
    result["entities"] = [{"entity": e["entity_group"], "word": e["word"]} for e in entities]
    del ner_pipe; torch.cuda.empty_cache()
    
    # 3. Summary (if long)
    if len(text.split()) > 100:
        summarizer = pipeline("summarization", device=DEVICE)
        result["summary"] = summarizer(text, max_length=50, min_length=20)[0]['summary_text']
        del summarizer; torch.cuda.empty_cache()
    
    return result
```
</details>

In [ ]:
# YOUR CODE HERE
# Create a function that takes customer feedback and returns:
# 1. Sentiment (positive/negative)
# 2. Key entities mentioned
# 3. A summary if the text is long

def analyze_customer_feedback(text: str) -> Dict[str, Any]:
    """
    Comprehensive customer feedback analysis.
    
    Args:
        text: Customer feedback text
        
    Returns:
        Dict with sentiment, entities, and summary
    """
    result = {
        "original_text": text,
        "sentiment": None,
        "entities": [],
        "summary": None
    }
    
    # TODO: Add sentiment analysis
    # TODO: Add NER
    # TODO: Add summarization if text is long (> 100 words)
    
    return result

# Test with a sample review
sample_feedback = """
I bought the iPhone 15 Pro from Apple Store in San Francisco last week. 
The sales representative John was incredibly helpful and knowledgeable. 
However, I'm disappointed with the battery life - it barely lasts a full day 
with normal usage. The camera quality is excellent though, especially for 
low-light photography. I've contacted Apple Support and they suggested a 
software update might help. Overall, it's a mixed experience but I'm hopeful 
the issues will be resolved.
"""

# Uncomment after implementing:
# analysis = analyze_customer_feedback(sample_feedback)
# print(json.dumps(analysis, indent=2))

---

## Common Mistakes

### Mistake 1: Not Cleaning Up Between Pipelines

```python
# Wrong: Memory keeps accumulating
pipe1 = pipeline("sentiment-analysis", device=0)
pipe2 = pipeline("ner", device=0)  # OOM on smaller GPUs!

# Right: Standard cleanup pattern
pipe1 = pipeline("sentiment-analysis", device=0)
# ... use pipe1 ...
del pipe1
gc.collect()  # Python garbage collection first
torch.cuda.empty_cache()  # Then release CUDA memory
pipe2 = pipeline("ner", device=0)  # Works!
```

### Mistake 2: Forgetting batch_size for Large Datasets

```python
# Wrong: Processes one by one (slow)
for text in large_list:
    result = pipe(text)

# Right: Batch processing (much faster!)
results = pipe(large_list, batch_size=32)
```

### Mistake 3: Using Wrong Device Index

```python
# Wrong: device="cuda" (string)
pipe = pipeline("sentiment-analysis", device="cuda")

# Right: device=0 (integer for GPU index, -1 for CPU)
pipe = pipeline("sentiment-analysis", device=0)
```

### Mistake 4: Not Using BFloat16 on DGX Spark

```python
# Wrong: Uses default FP32, wastes memory
pipe = pipeline("text-generation", device=0)

# Right: Use BF16 for Blackwell GPU (native support)
pipe = pipeline(
    "text-generation",
    device=0,
    torch_dtype=torch.bfloat16
)
```

---

## Checkpoint

You've learned:
- How to use 5 essential pipeline types
- How to benchmark pipeline performance
- Advanced pipelines: zero-shot classification, fill-mask
- When to use pipelines vs. manual model loading

---

## Further Reading

- [Pipeline Documentation](https://huggingface.co/docs/transformers/main_classes/pipelines)
- [Available Tasks](https://huggingface.co/tasks)
- [Inference Optimization](https://huggingface.co/docs/transformers/perf_infer_gpu_one)

---

## Cleanup

In [ ]:
import gc

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

print("\nLab 2.5.2 complete!")